In [9]:
###Import statements
import arcpy
import os
from arcpy import env

###Define the workspace
ws = arcpy.env.workspace = r"#Replace with workspace path
###Set GDB
gdb_path = r"#Replace with workspace path
arcpy.env.overwriteOutput = True

fclist = arcpy.ListFeatureClasses()
#No feature classes

#Copy data to project gdb
def copy_to_project(workspaces, Proj_database):
    workspaces = arcpy.ListWorkspaces("*","FileGDB")
    for w in workspaces:
        print(w)
        arcpy.env.workspace = w
        fcs = arcpy.ListFeatureClasses()
        fct = arcpy.ListTables()
        for fc in fcs:
            print(fc)
            if fc[4:] == "TaxPar":
                print("Tax Parcel Identified")
                arcpy.conversion.FeatureClassToGeodatabase(fc,Proj_database)
        for ft in fct:
            print(ft)
            if ft[4:] == "Assess":
                print("Assessor's Table Identified")
                arcpy.conversion.TableToGeodatabase(ft,Proj_database)
                        
#copy_to_project(ws, gdb_path)

#Check database
arcpy.env.workspace = gdb_path
cl = arcpy.ListFeatureClasses()
#print(cl)

def Join_Parcels_To_Tables():
    parcel_fc = arcpy.ListFeatureClasses()
    for pfc in parcel_fc:
        print(pfc)
        assessor_tbl = arcpy.ListTables()
        for tbl in assessor_tbl:
            print(tbl)
            if pfc[:4] == tbl[:4]:
                print("FC matched with Assessor's Table")
                print(pfc)
                x = arcpy.management.AddJoin(pfc,"LOC_ID",tbl,"LOC_ID")
                outfc = pfc[:4] + "_Parcels"
                arcpy.management.CopyFeatures(x,outfc)
#Join_Parcels_To_Tables()

### Merge parcels into one file
def mergeparcels():
    print("Initialize Parcel Merge")
    parcel_merge = "Cape_Ann_Parcel_Merge"
    cl = arcpy.ListFeatureClasses()
    filtered_list = [filt for filt in cl if 'Parcels' in filt]
    arcpy.management.Merge(filtered_list,parcel_merge)

#mergeparcels()

#Clip BioMap layers to Cape Ann using for loop; clipping to reduce processing time
study_area = "Cape_Ann_Parcel_Merge"
spatialLayers = ["Aquatic_Core_Buffer", "Wetland_Core_Buffers", "Coastal_Adaptation_Areas",
              "Tern_Foraging_Habitat", "Landscape_Blocks", "Aquatic_Core", "Wetland_Core",
                 "Priority_Natural_Communities_Core", "Forest_Core", "Vernal_Pool_Core",
                "Rare_Species_Core"]

def clip_biomap(layerList, studyArea):
    for lyr in layerList:
        new_lyr = lyr + "Clip"
        arcpy.analysis.Clip(lyr, studyArea, new_lyr)
        #arcpy.SpatialJoin_analysis(new_lyr,lyr, new_lyr + "_join")
        #arcpy.Buffer_analysis("East_Quabbin_Wetlands","Buffer_200",200,"FULL","ROUND","NONE")

#clip_biomap(spatialLayers, study_area)

#Intersect parcels and biomap layers
def biomap_parcels_intersect(layerList, studyArea):
    cpy = studyArea + "_Copy"
    arcpy.management.MakeFeatureLayer(studyArea, cpy)
    for lyr in layerList:
        newLyrName = lyr + "Clip"
        arcpy.management.SelectLayerByLocation(cpy,"INTERSECT", newLyrName,"","NEW_SELECTION")
        arcpy.management.CopyFeatures(cpy, lyr + "Selected_Parcels")
        

#biomap_parcels_intersect(spatialLayers, study_area)

#More information about MA building codes here: https://www.mass.gov/doc/property-type-classification-codes-non-arms-length-codes-and-sales-report-spreadsheet/download 
#multiuse_codes = list(range(0, 100))
#res_codes = list(range(100,132))
open_land_codes = list(range(200, 231))
vacant_land_codes = list(range(390, 394))
non_pr_land = [720]
study_codes = open_land_codes + vacant_land_codes + non_pr_land

###filter_study_parcels takes a long time to execute; shortening the layerList
###to a couple layers at a time might be preferrable
def filter_study_parcels(layerList, filterCodes) :
    for lyr in layerList:
        new_layer_name = lyr + "Selected_Parcels"
        print(new_layer_name)
        cpy = new_layer_name + "_copy"
        fields = arcpy.ListFields(new_layer_name)
        use_fields = []
        for field in fields:
            if 'USE_CODE' in field.name:
                use_fields.append(field.name)
        #print(use_fields)
        arcpy.management.MakeFeatureLayer(new_layer_name, cpy)
        for uf in use_fields:
            for val in filterCodes:
                where_cl = "{} = '{}'".format(uf, val)
                #print(where_cl)
                arcpy.management.SelectLayerByAttribute(cpy,"ADD_TO_SELECTION", where_cl)
        arcpy.management.CopyFeatures(cpy, lyr + "_Filtered_Parcels")
        arcpy.management.SelectLayerByAttribute(cpy, "CLEAR_SELECTION")

#filter_study_parcels(spatialLayers, study_codes)

#Clean up excess null fields from the intersect; takes long time to execute
#Based on: https://gis.stackexchange.com/questions/204684/delete-empty-and-null-fields-using-arcpy 
def DelFields():
    fcs = arcpy.ListFeatureClasses()
    study_fcs = []
    for c in fcs:
        if 'Filtered_Parcels' in c:
            study_fcs.append(c)    
    print(study_fcs)
    for fc in study_fcs:
        # Test whether the field is required--rather than testing the field type
        fieldList = [f.name for f in arcpy.ListFields(fc) if not f.required]

        # Iterate over the fields
        for field in fieldList:
            keep = False
            with arcpy.da.SearchCursor(fc, field) as cursor:
                for row in cursor:
                    v = row[0]
                    if v is not None or \
                      (isinstance(v, str) \
                      and (v.lower.strip() != ''\
                        or v != '<null>')):
                        keep = True
                        break
            if keep is False:
                arcpy.management.DeleteField(fc, field)

DelFields()

['All_Study_Parcels_Merge']
